In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/04 18:44:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.createOrReplaceTempView('green')

In [9]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_of_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [10]:
df_green_revenue.show()

+-------------------+----+------------------+-----------------+
|               hour|zone|            amount|number_of_records|
+-------------------+----+------------------+-----------------+
|2020-01-01 00:00:00|   7| 769.7299999999997|               45|
|2020-01-01 00:00:00|  17|            195.03|                9|
|2020-01-01 00:00:00|  18|               7.8|                1|
|2020-01-01 00:00:00|  22|              15.8|                1|
|2020-01-01 00:00:00|  24|              87.6|                3|
|2020-01-01 00:00:00|  25|             531.0|               26|
|2020-01-01 00:00:00|  29|              61.3|                1|
|2020-01-01 00:00:00|  32| 68.94999999999999|                2|
|2020-01-01 00:00:00|  33|317.27000000000004|               11|
|2020-01-01 00:00:00|  35|129.95999999999998|                5|
|2020-01-01 00:00:00|  36|295.34000000000003|               11|
|2020-01-01 00:00:00|  37|175.67000000000002|                6|
|2020-01-01 00:00:00|  38| 98.7899999999

In [22]:
df_green_revenue.repartition(20).write.parquet('data/report/revenue/green', mode='overwrite')

In [14]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [16]:
df_yellow.createOrReplaceTempView('yellow')

In [19]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_of_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [20]:
df_yellow_revenue.show()

+-------------------+----+------------------+-----------------+
|               hour|zone|            amount|number_of_records|
+-------------------+----+------------------+-----------------+
|2020-01-03 19:00:00| 142| 6023.090000000007|              403|
|2020-01-26 14:00:00| 239| 6541.650000000012|              437|
|2020-01-09 01:00:00| 100|            653.56|               37|
|2020-01-31 18:00:00| 237|12689.400000000034|              810|
|2020-01-04 03:00:00| 246| 2092.539999999999|              121|
|2020-01-14 09:00:00| 239| 4882.360000000001|              298|
|2020-01-27 16:00:00| 162| 7989.980000000011|              452|
|2020-01-17 20:00:00| 170|6884.1900000000105|              407|
|2020-01-23 15:00:00| 142| 5378.830000000004|              341|
|2020-01-27 06:00:00|  24|            536.49|               23|
|2020-01-01 04:00:00| 170| 2306.199999999999|              135|
|2020-01-05 12:00:00|  68|3495.9599999999973|              235|
|2020-01-13 17:00:00| 107|4066.679999999

In [23]:
df_yellow_revenue.repartition(20).write.parquet('data/report/revenue/yellow', mode='overwrite')

In [38]:
# Instead of doing chain of computation do first part, save the result & in next step read the results of the first part
# This process is known as materialising

df_green_revenue = spark.read.parquet('data/report/revenue/green/')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow/')

In [66]:
df_green_revenue

DataFrame[hour: timestamp, zone: int, amount: double, number_of_records: bigint]

In [67]:
df_yellow_revenue

DataFrame[hour: timestamp, zone: int, amount: double, number_of_records: bigint]

In [39]:
df_green_revenue_tmp = df_green_revenue \
                        .withColumnRenamed('amount', 'green_amount') \
                        .withColumnRenamed('number_of_records', 'green_number_of_records')

df_yellow_revenue_tmp = df_yellow_revenue \
                        .withColumnRenamed('amount', 'yellow_amount') \
                        .withColumnRenamed('number_of_records', 'yellow_number_of_records')

In [64]:
df_green_revenue_tmp

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_of_records: bigint]

In [65]:
df_yellow_revenue_tmp

DataFrame[hour: timestamp, zone: int, yellow_amount: double, yellow_number_of_records: bigint]

In [40]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [42]:
df_join.show()

+-------------------+----+------------------+-----------------------+------------------+------------------------+
|               hour|zone|      green_amount|green_number_of_records|     yellow_amount|yellow_number_of_records|
+-------------------+----+------------------+-----------------------+------------------+------------------------+
|2020-01-01 00:00:00|  22|              15.8|                      1|              null|                    null|
|2020-01-01 00:00:00|  25|             531.0|                     26|            324.35|                      16|
|2020-01-01 00:00:00|  55|129.29000000000002|                      4|              null|                    null|
|2020-01-01 00:00:00|  56|             99.69|                      3|              18.1|                       2|
|2020-01-01 00:00:00|  60|            160.04|                      6|57.620000000000005|                       2|
|2020-01-01 00:00:00|  61|            526.71|                     17|            146.64|

In [43]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [44]:
df_join = spark.read.parquet('data/report/revenue/total')

In [46]:
df_join.show()

+-------------------+----+------------------+-----------------------+------------------+------------------------+
|               hour|zone|      green_amount|green_number_of_records|     yellow_amount|yellow_number_of_records|
+-------------------+----+------------------+-----------------------+------------------+------------------------+
|2020-01-01 00:00:00|   4|              null|                   null|1004.3000000000002|                      57|
|2020-01-01 00:00:00|  15|              null|                   null|             34.09|                       1|
|2020-01-01 00:00:00|  35|129.95999999999998|                      5|              null|                    null|
|2020-01-01 00:00:00|  43|            107.52|                      6| 6539.510000000009|                     390|
|2020-01-01 00:00:00|  45|              null|                   null| 732.4800000000002|                      42|
|2020-01-01 00:00:00|  47|              13.3|                      1|               8.3|

In [55]:
!ls zones

_SUCCESS  part-00000-e993c72a-53da-4cd5-9cd9-eff7b831c60d-c000.snappy.parquet


In [56]:
df_zones = spark.read.parquet('zones/')

In [57]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [58]:
# Joining smaller table on bigger one
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [61]:
df_result.drop('LocationID', 'zone').show()

+-------------------+------------------+-----------------------+------------------+------------------------+---------+------------+
|               hour|      green_amount|green_number_of_records|     yellow_amount|yellow_number_of_records|  Borough|service_zone|
+-------------------+------------------+-----------------------+------------------+------------------------+---------+------------+
|2020-01-01 00:00:00|              null|                   null|1004.3000000000002|                      57|Manhattan| Yellow Zone|
|2020-01-01 00:00:00|              null|                   null|             34.09|                       1|   Queens|   Boro Zone|
|2020-01-01 00:00:00|129.95999999999998|                      5|              null|                    null| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|            107.52|                      6| 6539.510000000009|                     390|Manhattan| Yellow Zone|
|2020-01-01 00:00:00|              null|                   null| 732.4800000

In [62]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')